In [38]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [51]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.utils import pretty_qf
from requests import get
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 01 seconds to download 1000 candles


In [41]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=15,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([1, 3, 6]),
    sl_based_on_add_pct=np.array([0.01, 0.03, .06]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([.1, 0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2, 4]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [42]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 486
Total combinations of settings to test: 2,916
Total candles: 1,000
Total candles to test: 2,916,000


,ind_set_idx,dos_index,total_trades,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,57,62.0,-84.28,16.13,0.0,609.149,-842.7882,157.2118
1,0,111,68.0,-58.69,20.59,0.0,1055.007,-586.9053,413.0947
2,0,129,63.0,-56.09,22.22,0.0,856.561,-560.8809,439.1191
3,0,147,61.0,-40.03,22.95,0.0,928.464,-400.2973,599.7027
4,0,273,66.0,-56.46,21.21,0.0,1274.001,-564.6499,435.3501
5,0,291,70.0,-63.57,20.00,0.0,1196.528,-635.7407,364.2593
6,0,309,61.0,-47.10,22.95,0.0,1259.826,-471.0076,528.9924
7,0,381,63.0,-86.29,19.05,0.0,827.774,-862.8866,137.1134
8,0,399,64.0,-82.45,20.31,0.0,828.954,-824.5095,175.4905
9,0,435,73.0,-71.13,17.81,0.0,1430.949,-711.3392,288.6608


In [43]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [44]:
temp_df = order_records_df[
    ["order_status", "timestamp", "average_entry", "entry_price", "liq_price", "sl_price", "tp_price"]
]

In [45]:
entries_df = temp_df[temp_df["order_status"] == "EntryFilled"]

In [46]:
entries_list_df = entries_df.values[:, 1:].tolist()

In [47]:
entries_list = candles_np[:, 0].tolist()

In [48]:
final_list = []
for value in entries_list:
    if value != entries_list_df[0][0]:
        final_list.append([value])
    else:
        final_list.append(entries_list_df[0])
        del entries_list_df[0]

IndexError: list index out of range

In [50]:
data = {
    "entries": final_list,
    "candles": candles_np.tolist(),
}

In [52]:
res = requests.post('http://127.0.0.1:7779/plot', json=data)

# Convert response data to json
returned_data = res.json()

print(returned_data)

{'ok': True}
